Demand forecasting with BigQuery and TensorFlow
In this notebook, we will develop a machine learning model to predict the demand for taxi cabs in New York.
To develop the model, we will need to get historical data of taxicab usage. This data exists in BigQuery. Let's start by looking at the schema.

In [1]:
import gcp.bigquery as bq
import pandas as pd
import numpy as np

In [6]:
%%bigquery schema --table "nyc-tlc:green.trips_2015"

Analyzing taxicab demand
Let's pull the number of trips for each day in the 2015 dataset. We can use the BigQuery built-in Date-and-time function DAYOFYEAR (See BigQuery query reference ) for a full list of such functions.

In [7]:

%%sql
SELECT DAYOFYEAR(pickup_datetime) AS daynumber FROM [nyc-tlc:green.trips_2015] LIMIT 10

daynumber
120
150
150
150
150
89
89
89
89
90


Modular queries and Pandas dataframe
Let's use the total number of trips as our proxy for taxicab demand (other reasonable alternatives are total trip_distance or total fare_amount). It is possible to predict multiple variables using Tensorflow, but for simplicity, we will stick to just predicting the number of trips.
We will give our query a name 'taxiquery' and have it use an input variable '$YEAR'. We can then invoke the 'taxiquery' by giving it a YEAR. The to_dataframe() converts the BigQuery result into a Pandas dataframe.

In [8]:
%%sql --module taxiquery
SELECT daynumber, COUNT(*) AS numtrips FROM
    (SELECT DAYOFYEAR(pickup_datetime) AS daynumber FROM [nyc-tlc:green.trips_$YEAR])
GROUP BY daynumber ORDER BY daynumber

In [10]:
trips = bq.Query(taxiquery, YEAR=2015).to_dataframe()
trips[:7]

,daynumber,numtrips
0,1,62943
1,2,43410
2,3,53866
3,4,41602
4,5,41923
5,6,40646
6,7,47802
